In [93]:
from tqdm import tqdm
from yaml import safe_load
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

## Configuration

In [94]:
# Retrieve configuration file for elastic
with open('../../config/debug.config.yml') as config:
    config_file = safe_load(config)['elastic']

## Evaluation

In [95]:
# Initiate connection with Elasticsearch
es_client = Elasticsearch(
    f"{config_file['protocol']}://{config_file['host']}:{config_file['port']}",
    ca_certs='../../ca.crt',
    basic_auth=(config_file['user'], config_file['password'])
)

In [96]:
def retrieve_all_ids():
    mongo_ids = open('./data/mongo-ids.txt', mode='w+')
    
    for hit in tqdm(scan(es_client, index='apis', query={'query': {'match_all': {}}}), total=943254):
        mongo_ids.write(f"{hit['_source']['metadata']['mongo-id']}\n")
    
    mongo_ids.close()

retrieve_all_ids()

100%|██████████| 943254/943254 [17:51<00:00, 880.11it/s] 


In [97]:
def retrieve_documents(n: int = 1000):
    for i in range(n):
        pass